In [1]:
import math, os

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
current_dir = os.getcwd()
lesson_dir = current_dir
data_dir = current_dir + '/data/catsdogs'
print(data_dir)
print(current_dir)

/home/ubuntu/nbs/courses/deeplearning2/data/catsdogs
/home/ubuntu/nbs/courses/deeplearning2


In [5]:
%matplotlib inline

In [6]:
%cd $data_dir

#Set path to sample/ path to run on sample
path = data_dir
#path = data_dir + '/sample/'

test_path = data_dir + '/test/' 
results_path= data_dir + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

print (train_path)

/home/ubuntu/nbs/courses/deeplearning2/data/catsdogs
/home/ubuntu/nbs/courses/deeplearning2/data/catsdogs/train/


In [7]:
import tensorflow
import keras
from keras import backend as K

Using TensorFlow backend.


In [8]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras import optimizers

from keras.models import Model, Sequential
from keras.layers import Dense
from keras.layers import *

In [9]:
batch_size = 64

In [10]:
datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)
train_datagen = datagen.flow_from_directory(train_path, target_size = (224, 224), 
                                            class_mode = 'categorical', batch_size = batch_size)

valid_datagen = datagen.flow_from_directory(valid_path, target_size = (224, 224), 
                                            class_mode = 'categorical', batch_size = batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [11]:
base_model = ResNet50(weights = 'imagenet')

In [12]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [13]:
#last = base_model.output
last = base_model.layers[-2].output
preds = Dense(2, activation='softmax')(last)

new_model = Model(inputs=base_model.input, outputs = preds)


In [14]:
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [15]:
new_model.get_layer(index = 175) #Used trial and error in this case. Can also use code below

#layers = new_model.layers
#first_dense_index = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]

In [16]:
for layer in new_model.layers[:175]: 
    layer.trainable = False

In [17]:
adam = optimizers.Adam(lr = 0.001) 

new_model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [18]:
num_epochs = 30
num_data = 23000
num_valdata = 2000
#num_data = 600 # for samples
#num_valdata = 200 # for samples

In [19]:
steps = num_data/batch_size
valid_steps = num_valdata/batch_size

print(steps)
print(valid_steps)

359.375
31.25


In [20]:
from keras.callbacks import *

In [21]:
filepath = results_path + "wResnet.{epoch:02d}-{val_loss:.2f}.hdf5"

callback_list = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=2),
    ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)]

new_model.fit_generator(train_datagen, steps_per_epoch = steps, epochs = num_epochs, verbose = 2, 
                    validation_data = valid_datagen, validation_steps = valid_steps, callbacks = callback_list)

#new_model.save_weights(results_path + 'first_try.h5')



Epoch 1/30
Epoch 00001: val_loss improved from inf to 0.05297, saving model to /home/ubuntu/nbs/courses/deeplearning2/data/catsdogs/results/wResnet.01-0.05.hdf5
 - 251s - loss: 0.0958 - acc: 0.9622 - val_loss: 0.0530 - val_acc: 0.9790
Epoch 2/30
Epoch 00002: val_loss improved from 0.05297 to 0.05287, saving model to /home/ubuntu/nbs/courses/deeplearning2/data/catsdogs/results/wResnet.02-0.05.hdf5
 - 249s - loss: 0.0504 - acc: 0.9807 - val_loss: 0.0529 - val_acc: 0.9815
Epoch 3/30
Epoch 00003: val_loss did not improve
 - 249s - loss: 0.0457 - acc: 0.9822 - val_loss: 0.0566 - val_acc: 0.9785
Epoch 4/30
Epoch 00004: val_loss improved from 0.05287 to 0.05253, saving model to /home/ubuntu/nbs/courses/deeplearning2/data/catsdogs/results/wResnet.04-0.05.hdf5
 - 249s - loss: 0.0441 - acc: 0.9837 - val_loss: 0.0525 - val_acc: 0.9805
Epoch 5/30
Epoch 00005: val_loss improved from 0.05253 to 0.05184, saving model to /home/ubuntu/nbs/courses/deeplearning2/data/catsdogs/results/wResnet.05-0.05.hdf5